# Drivers of Weekly+
The purpose of this notebook is to identify the best leading indicator of Weekly+, then determine the behavioral, attitudinal, and perception drivers of Weekly+. It does this with live connections to SQL (where BEACH raw data is stored) and connections to the BEACH data model (the Power BI Semantic model). It then uses ChatGPT to summarize the results for ease of understanding by non-technical users.

Step 1: Initialize connections and load packages

Step 2: Identify which measure we should use as a leading indicator

Step 3: Get data from SQL

Step 4: Perform models to measure drivers

Step 5: Summarize results for non-technical users

## Initialization of the Model
Load Packages and Decide What to Model

#### Load Packages to be used by this program

In [1]:
import openai
from dotenv import load_dotenv
import os
from pyadomd import Pyadomd
import pyodbc
import pandas as pd
from openai import OpenAI
import re
import time
import os
import difflib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler  # Import the undersampler
import statsmodels.api as sm
from sqlalchemy import create_engine
import urllib.request
import json
from pptx import Presentation
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN


#### Decide which country and brand you want to analyze drivers

In [2]:
country_id = 'USA'
brand_id = 'br_0201'
drink_type = "SSD Regular"

#### Set Power BI Connection Details

In [3]:
# Connection details
dataset = 'BEACH'
port_number = 'powerbi://api.powerbi.com/v1.0/myorg/SIMON BEACH'
user_id = 'elimoore@coca-cola.com'
password = os.environ.get("KO_PASSWORD")
pbi_connection_string = f'Provider=MSOLAP;Data Source={port_number};Catalog={dataset};User ID={user_id};Password={password};'

#### Load OpenAI Key

In [4]:
# Initialize OpenAI client
#api_key = os.environ.get("OPENAI_API_KEY")
deployment = 'gpt-4'
apiversion = '2024-06-01'
api_key = os.environ.get("KO_GPT_API")

#### Set SQL Server Connection Details

In [5]:
# Connection information
server = 'syn-tccc-blc-use2-prod-01-ondemand.sql.azuresynapse.net'
database = 'beach_db'
username = 'elimoore@coca-cola.com'
password = os.environ.get("KO_PASSWORD")
driver = '{ODBC Driver 17 for SQL Server}'

# Connection string
connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Authentication=ActiveDirectoryPassword;Connection Timeout=30'


#### Identify Brand Name

In [10]:
# SQL query
sql_query = """
    /****** Script for SelectTopNRows command from SSMS  ******/
    SELECT distinct(brand) FROM [dm].[Dataset_Product]
      where brand_id = ?
"""

with pyodbc.connect(connection_string) as connection:
    cursor = connection.cursor()

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    brand_name_data = pd.read_sql(sql_query, connection, params=[brand_id])
    
    # Retrieve the top row value as a string
    if not brand_name_data.empty:
        brand_name = brand_name_data.iloc[0]['brand']
        print(brand_name)
    else:
        print("No data found.")

C:\Users\O52834\AppData\Local\Temp\ipykernel_16128\2170411904.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  brand_name_data = pd.read_sql(sql_query, connection, params=[brand_id])


Coca-Cola/ Coca-Cola Classic


#### Get Brand Logo

In [76]:
# SQL query
sql_query = """
    SELECT top 1 [Logo_URL]   
    FROM [dm].[Dataset_Product]
    where brand_id = ? and country_id=?
"""

with pyodbc.connect(connection_string) as connection:
    cursor = connection.cursor()

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    brand_name_data = pd.read_sql(sql_query, connection, params=[brand_id,country_id])
    
    # Retrieve the top row value as a string
    if not brand_name_data.empty:
        logo_url = brand_name_data.iloc[0]['Logo_URL']
        print(logo_url)
    else:
        print("No data found.")

C:\Users\O52834\AppData\Local\Temp\ipykernel_16128\387349674.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  brand_name_data = pd.read_sql(sql_query, connection, params=[brand_id,country_id])


https://map.cdn.ccnag.com/coca-cola-logo.png


#### Specify the location of the PPT Template File and Final Pasting Location

In [56]:
# File paths
template_path = r"C:\Users\O52834\OneDrive - The Coca-Cola Company\Documents\BEACH\W+ Modeling\Weekly_Plus_Drivers_Template.pptx"
output_path = fr"C:\Users\O52834\OneDrive - The Coca-Cola Company\Documents\BEACH\W+ Modeling\Iteration Outputs\{country_id}_{brand_id}.pptx"

## Prologue: Identifying best leading indicator of Weekly+
This script demonstrates the correlation between Weekly+ and key leading indicators like Past 7 Day Weekly+ and Past 7 Day Incidence

In [ ]:
# DAX query with dynamic drink type
dax_query = f"""
// DAX Query
DEFINE
    VAR __DS0FilterTable = 
        TREATAS({{"12MMT"}}, '3 Time Period'[Period_Type])

    VAR __DS0FilterTable2 = 
        TREATAS({{"RTD"}}, '4 Product'[RTD_NRTD])

    VAR __DS0FilterTable3 = 
        TREATAS({{"{drink_type}"}}, '4 Product'[Drink Type])

    VAR __DS0Core = 
        SUMMARIZECOLUMNS(
            '5 Demographics and Geography'[Country_ID],
            '3 Time Period'[Month_Name],
            '3 Time Period'[Month_MMT_ID],
            '4 Product'[Trademark],
            __DS0FilterTable,
            __DS0FilterTable2,
            __DS0FilterTable3,
            "WeeklyPlus", '1 Measures'[Weekly+ Incidence (% weighted)],
            "P7DWeeklyPlus", '1 Measures'[Past 7 Day Weekly+ Incidence (% weighted)],
            "P7D", '1 Measures'[Past 7 Day Drinkers (% weighted)],
            "Total_Drinks", '1 Measures'[Total Drinks]
        )

EVALUATE
    __DS0Core

ORDER BY
    '5 Demographics and Geography'[Country_ID],
    '3 Time Period'[Month_MMT_ID],
    '3 Time Period'[Month_Name],
    '4 Product'[Trademark]
"""

# Use Pyadomd context manager to handle the connection
with Pyadomd(pbi_connection_string) as conn:
    with conn.cursor().execute(dax_query) as cur:
        power_bi_data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

# Optionally, print the top 5 rows to confirm the data was retrieved correctly
print(power_bi_data.head(5))

In [ ]:
# Create a dictionary mapping old column names to new names
column_mapping = {
    '5 Demographics and Geography[Country_ID]': 'Country_ID',
    '3 Time Period[Month_Name]': 'Month_Name',
    '3 Time Period[Month_MMT_ID]': 'Month_MMT_ID',
    '4 Product[Trademark]': 'Trademark',
    '[WeeklyPlus]': 'WeeklyPlus',
    '[P7DWeeklyPlus]': 'P7DWeeklyPlus',
    '[P7D]': 'P7D',
    '[Total_Drinks]': 'Total Drinks'
}

# Rename the columns using the mapping dictionary
df_copy = power_bi_data.copy()
df_copy.rename(columns=column_mapping, inplace=True)

# Convert categorical variables into numerical values if needed
df_copy['Country_ID'] = pd.factorize(df_copy['Country_ID'])[0]
df_copy['Trademark'] = pd.factorize(df_copy['Trademark'])[0]

# Convert columns to numeric, coerce errors to NaN
df_copy['P7D'] = pd.to_numeric(df_copy['P7D'], errors='coerce')
df_copy['P7DWeeklyPlus'] = pd.to_numeric(df_copy['P7DWeeklyPlus'], errors='coerce')
df_copy['WeeklyPlus'] = pd.to_numeric(df_copy['WeeklyPlus'], errors='coerce')
df_copy['Total Drinks'] = pd.to_numeric(df_copy['Total Drinks'], errors='coerce')

# Drop rows with any NaN values in the relevant columns
df_copy = df_copy.dropna(subset=['P7D', 'P7DWeeklyPlus', 'WeeklyPlus', 'Total Drinks'])

# Calculate Pearson correlation between WeeklyPlus and P7D
correlation_p7d = df_copy['WeeklyPlus'].corr(df_copy['P7D'])
# Calculate Pearson correlation between WeeklyPlus and P7DWeeklyPlus
correlation_p7dweeklyplus = df_copy['WeeklyPlus'].corr(df_copy['P7DWeeklyPlus'])
# Calculate Pearson correlation between WeeklyPlus and Total Drinks
correlation_totaldrinks = df_copy['WeeklyPlus'].corr(df_copy['Total Drinks'])
# Calculate Pearson correlation between P7D and Total Drinks
correlation_drinks_p7d = df_copy['P7D'].corr(df_copy['Total Drinks'])
# Calculate Pearson correlation between P7D and Total Drinks
correlation_drinks_p7dw = df_copy['P7DWeeklyPlus'].corr(df_copy['Total Drinks'])

# Print the correlation coefficients
print(f"Correlation between WeeklyPlus and P7D: {correlation_p7d:.4f}")
print(f"Correlation between WeeklyPlus and P7DWeeklyPlus: {correlation_p7dweeklyplus:.4f}")
print(f"Correlation between WeeklyPlus and Total Drinks: {correlation_totaldrinks:.4f}")
print(f"Correlation between P7D and Total Drinks: {correlation_drinks_p7d:.4f}")
print(f"Correlation between P7D WeeklyPlus and Total Drinks: {correlation_drinks_p7dw:.4f}")

In [ ]:
#Run a simple linear regression to show how strong the relationship is between P7D Weekly+ and Weekly+

# Define the target variable
y = df_copy['WeeklyPlus']
y2 = df_copy['P7D']

# Model 1: WeeklyPlus = f(Country_ID, Trademark, P7D)
X1 = df_copy[['Country_ID', 'Trademark', 'P7D']]
X1 = sm.add_constant(X1)  # Adds a constant term to the predictor
model1 = sm.OLS(y, X1).fit()
r2_model1 = model1.rsquared

# Model 2: WeeklyPlus = f(Country_ID, Trademark, P7DWeeklyPlus)
X2 = df_copy[['Country_ID', 'Trademark', 'P7DWeeklyPlus']]
X2 = sm.add_constant(X2)  # Adds a constant term to the predictor
model2 = sm.OLS(y, X2).fit()
r2_model2 = model2.rsquared

# Model 3: WeeklyPlus = f(Country_ID, Trademark, TotalDrinks)
X2 = df_copy[['Country_ID', 'Trademark', 'Total Drinks']]
X2 = sm.add_constant(X2)  # Adds a constant term to the predictor
model2 = sm.OLS(y, X2).fit()
r2_model3 = model2.rsquared

# Model 4: TotalDrinks = f(Country_ID, Trademark, P7D)
X2 = df_copy[['Country_ID', 'Trademark', 'Total Drinks']]
X2 = sm.add_constant(X2)  # Adds a constant term to the predictor
model2 = sm.OLS(y2, X2).fit()
r2_model4 = model2.rsquared

print(f"Provided the country, trademark, and P7D or P7DWeeklyPlus, we can confidently estimate Weekly+.")
print(f"R² for Equation 1 (WeeklyPlus = f(Country_ID, Trademark, P7D)): {r2_model1:.4f}")
print(f"R² for Equation 2 (WeeklyPlus = f(Country_ID, Trademark, P7DWeeklyPlus)): {r2_model2:.4f}")
print(f"R² for Equation 3 (WeeklyPlus = f(Country_ID, Trademark, Total Drinks)): {r2_model3:.4f}")
print(f"R² for Equation 4 (Drinks = f(Country_ID, Trademark, P7D)): {r2_model4:.4f}")

## Get Data for Modeling Weekly+
Using SQL and Azure, acquire the BEACH data for modeling

#### Diary Data
This script connects live to the BEACH SQL Server and pulls down relevant diary data for the particular brand and country

In [11]:
# SQL query
sql_query = """
    /****** Script for SelectTopNRows command from SSMS  ******/
    SELECT [Respondent_ID]
          ,right(Month_ID,2) as Month_of_Year
          ,case
            when [Frequency_Name] = 'Multiple times per week' then 1
            when [Frequency_Name] = 'Once a week' then 1
            when [Frequency_Name] = 'Daily' then 1
            else 0 end Frequency
          ,CASE
    		when [Day_Name] = 'Saturday' or [Day_Name] = 'Sunday' then 1 else 0 end Weekend
          ,[DayPart_Name]
          ,[Where_Name]
          ,[Who_Group_HL]
          ,[Bought_Name]
          ,[IC/FC-SS/MS]
          ,[Channel]
          ,[Reason_Energize]
          ,[Reason_Celebrate]
          ,[Reason_Stay]
          ,[Reason_Rehydrate]
          ,[Reason_Cheer]
          ,[Reason_Close]
          ,[Reason_Cool]
          ,[Reason_Nutritious]
          ,[Reason_Focus]
          ,[Reason_Complement]
          ,[Reason_Confident]
          ,[Reason_Taste]
          ,[Reason_Reward]
          ,[Reason_Restore]
          ,[Reason_Performance]
          ,[Reason_Other]
          ,[High-Level-Occasions]
      FROM [dm].[Dataset_Diary_SS]
      where country_id= ? and brand_id= ?
"""

with pyodbc.connect(connection_string) as connection:
    cursor = connection.cursor()

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    diarydata = pd.read_sql(sql_query, connection, params=[country_id, brand_id])
    df = diarydata

C:\Users\O52834\AppData\Local\Temp\ipykernel_16128\2999501368.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  diarydata = pd.read_sql(sql_query, connection, params=[country_id, brand_id])


#### Equity/Imagery Data
This script connects to SQL Server and pulls down equity data for the particular brand and country

In [12]:
# SQL query
sql_query = """
    /****** Script for SelectTopNRows command from SSMS  ******/
    WITH a AS (
    SELECT 
        index_resp,
        Image_Name,
        Country_ID,
        Brand_ID
    FROM 
        dm.Dataset_Imagery
	WHERE 
		Country_ID = ? 
		AND Brand_ID = ?
	),

b as(

SELECT 
    index_resp,
    MAX(CASE WHEN Image_Name = 'Encourages me to try new things' THEN 1 ELSE 0 END) AS [Encourages me to try new things],
    MAX(CASE WHEN Image_Name = 'Is a youthful brand' THEN 1 ELSE 0 END) AS [Is a youthful brand],
    MAX(CASE WHEN Image_Name = 'Is a brand for everyone' THEN 1 ELSE 0 END) AS [Is a brand for everyone],
    MAX(CASE WHEN Image_Name = 'Has a flavor I enjoy' THEN 1 ELSE 0 END) AS [Has a flavor I enjoy],
    MAX(CASE WHEN Image_Name = 'Shares my values' THEN 1 ELSE 0 END) AS [Shares my values],
    MAX(CASE WHEN Image_Name = 'Offers quality I can trust' THEN 1 ELSE 0 END) AS [Offers quality I can trust],
    MAX(CASE WHEN Image_Name = 'Cares about the environment' THEN 1 ELSE 0 END) AS [Cares about the environment],
    MAX(CASE WHEN Image_Name = 'Is easy to find in stores' THEN 1 ELSE 0 END) AS [Is easy to find in stores],
    MAX(CASE WHEN Image_Name = 'Has a unique taste' THEN 1 ELSE 0 END) AS [Has a unique taste],
    MAX(CASE WHEN Image_Name = 'Gives me an energy boost' THEN 1 ELSE 0 END) AS [Gives me an energy boost],
    MAX(CASE WHEN Image_Name = 'Goes well with food' THEN 1 ELSE 0 END) AS [Goes well with food],
    MAX(CASE WHEN Image_Name = 'Brings me happiness' THEN 1 ELSE 0 END) AS [Brings me happiness],
    MAX(CASE WHEN Image_Name = 'Is available in convenient packaging' THEN 1 ELSE 0 END) AS [Is available in convenient packaging],
    MAX(CASE WHEN Image_Name = 'Makes moments special' THEN 1 ELSE 0 END) AS [Makes moments special],
    MAX(CASE WHEN Image_Name = 'Has a refreshing taste' THEN 1 ELSE 0 END) AS [Has a refreshing taste],
    MAX(CASE WHEN Image_Name = 'Helps keep me going' THEN 1 ELSE 0 END) AS [Helps keep me going],
    MAX(CASE WHEN Image_Name = 'Has the right amount of calories' THEN 1 ELSE 0 END) AS [Has the right amount of calories],
    MAX(CASE WHEN Image_Name = 'Gives me confidence to be myself' THEN 1 ELSE 0 END) AS [Gives me confidence to be myself]
FROM 
    a

GROUP BY
	index_resp),

c as (
	SELECT 
			index_resp
		  ,[Familiarity_Name]
		  ,[Cons_Name]
		  ,[Affinity_Name]
		  ,[Cons_Segment]
		  ,[Unique_Name]
		  ,[MNeeds_Name]
		  ,[Dynamic_Name]
		  ,[Price_Name]
		  ,[Worth_Name]
	  FROM [dm].[Dataset_Equity]

	  where country_id= ? and brand_id= ?
  ),

  d1 as (

	  SELECT index_resp
		  ,case
			when [Frequency_Name] = 'Multiple times per week' then 1
			when [Frequency_Name] = 'Once a week' then 1
			when [Frequency_Name] = 'Daily' then 1
			else 0 end Frequency
	  FROM [dm].[Dataset_Diary_SS]

	  where country_id= ? and brand_id= ?

	),

d as (

	  SELECT index_resp, max(Frequency) as Frequency
	  FROM d1

	  group by index_resp


)

		Select 
			c.index_resp
			,d.Frequency
			,c.[Familiarity_Name]
			,c.[Cons_Name]
			,c.[Affinity_Name]
			,c.[Cons_Segment]
			,c.[Unique_Name]
			,c.[MNeeds_Name]
			,c.[Dynamic_Name]
			,c.[Price_Name]
			,c.[Worth_Name]
			, b.*
		from c


		left join b on b.index_resp = c.index_resp

		left join d on d.index_resp = c.index_resp

		left join dm.Dataset_Metrics q on c.index_resp = q.index_resp

		where q.survey_flag<>'Equity'

"""

with pyodbc.connect(connection_string) as connection:
    cursor = connection.cursor()

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    equitydata = pd.read_sql(sql_query, connection, params=[country_id, brand_id, country_id, brand_id, country_id, brand_id])

C:\Users\O52834\AppData\Local\Temp\ipykernel_16128\1980386704.py:117: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  equitydata = pd.read_sql(sql_query, connection, params=[country_id, brand_id, country_id, brand_id, country_id, brand_id])


## Creating Driver Models
Using Random Forest, create models to understand what makes someone become Weekly+

#### Diary Drivers
This script runs a random forest to see what the primary behavioral variables contribute to users becoming weekly+

In [13]:
# Preprocess the data
X = diarydata.drop(columns=['Respondent_ID', 'Frequency'])
y = diarydata['Frequency']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing for numerical data (standard scaling)
numerical_transformer = StandardScaler()

# Preprocessing for categorical data (one-hot encoding)
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = RandomForestClassifier(n_estimators=100, random_state=0)

# Create and evaluate the pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Feature importance
feature_names = list(numerical_cols) + list(clf.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_cols))
importances = clf.named_steps['model'].feature_importances_

# Combine feature names and importances into a DataFrame
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

              precision    recall  f1-score   support

           0       0.30      0.10      0.15      1242
           1       0.89      0.97      0.93      9533

    accuracy                           0.87     10775
   macro avg       0.60      0.54      0.54     10775
weighted avg       0.82      0.87      0.84     10775



#### Quantifying Impact of Changes in Behavior on Weekly+
This script determines the top behavioral drivers and the impact of changing them on someone's likelihood of becoming Weekly+

In [14]:
# PDP analysis for all features without plotting
results = []
X_transformed = clf.named_steps['preprocessor'].transform(X)

for feature_name in feature_names:
    feature_index = feature_names.index(feature_name)
    
    # Generate Partial Dependence values for the current feature without plotting
    pdp = partial_dependence(clf.named_steps['model'], X_transformed, [feature_index], grid_resolution=50)
    
    # Extract the averaged predicted probabilities
    pdp_values = pdp['average'].ravel()
    
    # Find the change in probability when moving from min to max value of the feature
    min_value_prob = pdp_values[0]
    max_value_prob = pdp_values[-1]
    prob_change = max_value_prob - min_value_prob
    
    # Convert to percentages
    min_value_prob_percent = min_value_prob * 100
    max_value_prob_percent = max_value_prob * 100
    prob_change_percent = prob_change * 100
    
    result = {
        'Feature': feature_name,
        'Min Value Predicted Probability (%)': min_value_prob_percent,
        'Max Value Predicted Probability (%)': max_value_prob_percent,
        'Change in Predicted Probability (%)': prob_change_percent
    }
    results.append(result)

# Convert results to a DataFrame for easier visualization
pdp_analysis_df = pd.DataFrame(results)

# Merge feature_importance_df with pdp_analysis_df on 'Feature' column
feature_importance_df = feature_importance_df.merge(pdp_analysis_df[['Feature', 'Change in Predicted Probability (%)']], on='Feature', how='left')

# Sort feature_importance_df by 'Change in Predicted Probability (%)' in descending order
feature_importance_df = feature_importance_df.sort_values(by='Change in Predicted Probability (%)', ascending=False)

# Display the most important features with added 'Change in Predicted Probability (%)'
print(feature_importance_df.head(30))

print(pdp_analysis_df.head(10))

# Filter out features with negative change and sort by positive change
positive_changes_df = pdp_analysis_df[pdp_analysis_df['Change in Predicted Probability (%)'] > 0]
top_positive_changes_df = positive_changes_df.sort_values(by='Change in Predicted Probability (%)', ascending=False).head(10)


interpretations = ""

# Interpretations for the top 5 features with positive changes
for index, row in top_positive_changes_df.iterrows():
    feature_name = row['Feature']
    min_value_prob_percent = row['Min Value Predicted Probability (%)']
    max_value_prob_percent = row['Max Value Predicted Probability (%)']
    prob_change_percent = row['Change in Predicted Probability (%)']
    
    interpretation = (
        f"Feature: {feature_name}\n"
        f"Change in predicted probability: +{prob_change_percent:.2f}%\n"
        f"For example, moving the perception of '{feature_name}' from a low level to a high level can increase the likelihood of frequent usage (Frequency = 1) by {prob_change_percent:.2f}%.\n"
    )
    
    # Append the interpretation to the string
    interpretations += interpretation
    
    print(f"Feature: {feature_name}")
    print(f"Min value predicted probability: {min_value_prob_percent:.2f}%")
    print(f"Max value predicted probability: {max_value_prob_percent:.2f}%")
    print(f"Change in predicted probability: +{prob_change_percent:.2f}%")
    print(f"For example, moving the perception of '{feature_name}' from a low level to a high level can increase the likelihood of frequent usage (Frequency = 1) by {prob_change_percent:.2f}%.")
    print(f"Our model suggests a probability increase from {min_value_prob_percent:.2f}% to {max_value_prob_percent:.2f}% when the feature value is maximized.")
    print()

diary_insight = f"The following are insights from the behavior (diary) drivers of Weekly+:\n\n{interpretations}\n\n"

diaryprompt = f"The following are the top features most important for predicting [Frequency] = 1 in a RandomForest model based on diary (consumer behavior) data:\n\n{feature_importance_df.head(10).to_string(index=False)}\n\n."

                                              Feature  Importance  \
27                     Bought_Name_I bought it myself    0.019617   
26                                 Where_Name_At Home    0.020199   
33  Channel_Grocery store (not a chain)/ Independe...    0.014928   
22  Channel_Supermarket/ Hypermarket/ warehouse store    0.020755   
12                                  IC/FC-SS/MS_IC-SS    0.024729   
5          DayPart_Name_Midday (between 11am and 2pm)    0.030083   
2                         High-Level-Occasions_Eating    0.031560   
18                                 Who_Group_HL_Alone    0.021841   
45   DayPart_Name_Early Morning (between 6am and 8am)    0.007458   
44              High-Level-Occasions_Starting The Day    0.008069   
31    DayPart_Name_Mid Morning (between 8am and 11am)    0.016775   
51  Bought_Name_Not applicable - drink was made at...    0.001681   
37    Channel_Convenience stores/ Gas/ Petrol Station    0.012731   
52                                

#### Equity/Imagery Drivers
This script runs a random forest to see what attitudinal/perception variables contribute to users becoming weekly+

In [15]:
# Fill missing values in the 'Frequency' column with 0s
equitydata['Frequency'] = equitydata['Frequency'].fillna(0)

# Preprocess the data
X = equitydata.drop(columns=['index_resp', 'Frequency'])  # Adjust with correct column names
y = equitydata['Frequency']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing for numerical data (standard scaling)
numerical_transformer = StandardScaler()

# Preprocessing for categorical data (one-hot encoding)
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply undersampling to the training data
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)  # Initialize RandomUnderSampler
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)  # Resample only training data

# Define the model
model = RandomForestClassifier(n_estimators=100, random_state=0)

# Create the pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

# Fit the model on the resampled training data
clf.fit(X_train_resampled, y_train_resampled)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Feature importance
feature_names = list(numerical_cols) + list(clf.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_cols))
importances = clf.named_steps['model'].feature_importances_

# Combine feature names and importances into a DataFrame
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print(feature_importance_df)

              precision    recall  f1-score   support

         0.0       0.94      0.70      0.80      1523
         1.0       0.37      0.80      0.51       342

    accuracy                           0.72      1865
   macro avg       0.66      0.75      0.65      1865
weighted avg       0.83      0.72      0.75      1865

                              Feature  Importance
18  Familiarity_Name_Drink most often    0.051077
23             Familiarity_Name_Tried    0.045453
37             Cons_Segment_Intenders    0.041640
29         Affinity_Name_+3 I love it    0.039802
11                Brings me happiness    0.037265
..                                ...         ...
64                    Dynamic_Name_NA    0.000027
48                     Unique_Name_NA    0.000024
38                    Cons_Segment_NA    0.000017
56                     MNeeds_Name_NA    0.000002
36                   Affinity_Name_NA    0.000000

[77 rows x 2 columns]


#### Quantifying Impact of Changes in Attitudes/Perceptions on Weekly+
Determine the top attitudinal/perception drivers impact on Weekly+

In [16]:
# PDP analysis for all features without plotting
results = []
X_transformed = clf.named_steps['preprocessor'].transform(X)

for feature_name in feature_names:
    feature_index = feature_names.index(feature_name)
    
    # Generate Partial Dependence values for the current feature without plotting
    pdp = partial_dependence(clf.named_steps['model'], X_transformed, [feature_index], grid_resolution=50)
    
    # Extract the averaged predicted probabilities
    pdp_values = pdp['average'].ravel()
    
    # Find the change in probability when moving from min to max value of the feature
    min_value_prob = pdp_values[0]
    max_value_prob = pdp_values[-1]
    prob_change = max_value_prob - min_value_prob
    
    # Convert to percentages
    min_value_prob_percent = min_value_prob * 100
    max_value_prob_percent = max_value_prob * 100
    prob_change_percent = prob_change * 100
    
    result = {
        'Feature': feature_name,
        'Min Value Predicted Probability (%)': min_value_prob_percent,
        'Max Value Predicted Probability (%)': max_value_prob_percent,
        'Change in Predicted Probability (%)': prob_change_percent
    }
    results.append(result)

# Convert results to a DataFrame for easier visualization
pdp_analysis_df = pd.DataFrame(results)

# Merge feature_importance_df with pdp_analysis_df on 'Feature' column
feature_importance_df = feature_importance_df.merge(pdp_analysis_df[['Feature', 'Change in Predicted Probability (%)']], on='Feature', how='left')

# Sort feature_importance_df by 'Change in Predicted Probability (%)' in descending order
feature_importance_df = feature_importance_df.sort_values(by='Change in Predicted Probability (%)', ascending=False)

# Display the most important features with added 'Change in Predicted Probability (%)'
print(feature_importance_df.head(30))

print(pdp_analysis_df.head(10))

# Filter out features with negative change and sort by positive change
positive_changes_df = pdp_analysis_df[pdp_analysis_df['Change in Predicted Probability (%)'] > 0]
top_positive_changes_df = positive_changes_df.sort_values(by='Change in Predicted Probability (%)', ascending=False).head(10)

interpretations = ""

# Interpretations for the top 5 features with positive changes
for index, row in top_positive_changes_df.iterrows():
    feature_name = row['Feature']
    min_value_prob_percent = row['Min Value Predicted Probability (%)']
    max_value_prob_percent = row['Max Value Predicted Probability (%)']
    prob_change_percent = row['Change in Predicted Probability (%)']
    
    interpretation = (
        f"Feature: {feature_name}\n"
        f"Change in predicted probability: +{prob_change_percent:.2f}%\n"
        f"For example, moving the perception of '{feature_name}' from a low level to a high level can increase the likelihood of frequent usage (Frequency = 1) by {prob_change_percent:.2f}%.\n"
    )
    
    # Append the interpretation to the string
    interpretations += interpretation
    
    print(f"Feature: {feature_name}")
    print(f"Min value predicted probability: {min_value_prob_percent:.2f}%")
    print(f"Max value predicted probability: {max_value_prob_percent:.2f}%")
    print(f"Change in predicted probability: +{prob_change_percent:.2f}%")
    print(f"For example, moving the perception of '{feature_name}' from a low level to a high level can increase the likelihood of frequent usage (Frequency = 1) by {prob_change_percent:.2f}%.")
    print(f"Our model suggests a probability increase from {min_value_prob_percent:.2f}% to {max_value_prob_percent:.2f}% when the feature value is maximized.")
    print()

equity_insight = f"The following are insights from the equity/imagery drivers of Weekly+:\n\n{interpretations}\n\n"

equityprompt = f"The following are the top features most important for predicting [Frequency] = 1 in a RandomForest model based on consumer attitude and perception data:\n\n{feature_importance_df.head(10).to_string(index=False)}\n\n."

                                       Feature  Importance  \
0            Familiarity_Name_Drink most often    0.051077   
3                   Affinity_Name_+3 I love it    0.039802   
2                       Cons_Segment_Intenders    0.041640   
5        Cons_Name_It would be my first choice    0.030347   
6          MNeeds_Name_7 Meets needs very well    0.027698   
29                               Unique_Name_6    0.015950   
52                                Price_Name_3    0.005104   
4                          Brings me happiness    0.037265   
11                         Helps keep me going    0.023415   
10                        Has a flavor I enjoy    0.023649   
18            Familiarity_Name_Drink regularly    0.019511   
34                              Dynamic_Name_6    0.014048   
15            Gives me confidence to be myself    0.020976   
13                  Offers quality I can trust    0.022316   
14                      Has a refreshing taste    0.022023   
8       

## ChatGPT Response
Provide a final summary of both equity and diary drivers combined, brought to you by GPT

#### Context
Provide context to GPT to support its interpretation of the model results

In [29]:
# Descript the structure of the prompt
prompt_structure = (
    "I have results from a model aimed at helping marketers understand how to make people drink their brands every single week based on Diary (behavior) and Equity (attitudes/perceptions) data. The following prompt is structured as follows:"
    "\n- A summary of context about the question to be answered, meaning of variables, and an example output that can help guide the structure of the final response."
    "\n- The results of the diary-specific random forest model that help explain what makes people drink every single week (weekly+)."
    "\n- A few key insights based on the Diary-specific random forest results."
    "\n- The results of the equity-specific random forest model that help explain what makes people drink every single week (weekly+)."
    "\n- A few key insights based on the equity-specific random forest results."
    "\n- Always structure the response with three sections split with the following headers: '#### Behavior Factors', '#### Equity Factors', '#### Recommendations'"
    "\n")


# Create a context string for ChatGPT
context = (
    "CONTEXT, DEFINITIONS, AND EXAMPLES: Below are important context, considerations, variable definitions, and a response example to consider prior to reviewing data and insights."
    "\n- Where Frequency = 1, the user consumes every week. This is referred to as Weekly+."
    "\n- While variable importance means the variable is meaningful, the % change in probability will be most interesting when referencing opportunities to size big opportunities."
    "\n- The equity model uses variables describing users' 'behaviors', 'attitudes' and 'perceptions' about the brand."
    "\n- Where imagery = 1, the user agrees that the brand has this imagery."
    "\n- The attitude questions are the ones with columns ending in '_name' and are scaled answers, generally from something like -3 (I hate it) to +3 (I love it)."
    "\n- The behavior model uses variables from a consumer diary, where we get context around every single drink they record."
    "\n- Consider merging related fields together into a single recommendation. Example, if there are multiple channels that are highly impactful, you could combine them into a single recommendation about channel."
    "\n\nHere are short descriptions of what each feature/variable means:"
    
    "\nAttitude/Preferences (Equity/Imagery) Feature Definitions:"
    "\n- Familiarity_Name: A field containing the response from a 6-point-scale to the question 'How familiar are you with each of these brands?'"
    "\n- Cons_Name: A field containing the response from a 4-point-scale to the question 'How likely are you to consider choosing each of these brands the next time you drink [insert category]?' from 'It would be my first choice' to 'I would not consider it.'"
    "\n- Affinity_Name: A field containing the response from a 7-point-scale to the question 'How do you feel about each brand?'"
    "\n- Unique_Name: A field containing the response from a 3-point-scale to the question 'Is this brand worth more or less than it costs?'"
    "\n- MNeeds_Name: A field containing the response from a 7-point-scale to the prompt 'Drag each brand onto the scale to show how well each brand delivers the main things you need from a [insert category name].'"
    "\n- Dynamic_Name: A field containing the response from a 7-point-scale to the prompt 'Drag each brand onto the scale to show how much it sets trends.'"
    "\n- Price_Name: A field containing the response from a 7-point-scale to the prompt 'Drag each brand onto the scale to show how much you think it costs.'"
    "\n- Worth_Name: A field containing the response from a 7-point-scale to the prompt 'Drag each brand onto the scale to show how different it seems from other brands of [insert category].'"
    "\n- Encourages me to try new things: A 1 when the consumer says they agree with the statement: 'Encourages me to try new things.'"
    "\n- Is a youthful brand: A 1 when the consumer says they agree with the statement: 'Is a youthful brand.'"
    "\n- Is a brand for everyone: A 1 when the consumer says they agree with the statement: 'Is a brand for everyone.'"
    "\n- Has a flavor I enjoy: A 1 when the consumer says they agree with the statement: 'Has a flavor I enjoy.'"
    "\n- Shares my values: A 1 when the consumer says they agree with the statement: 'Shares my values.'"
    "\n- Offers quality I can trust: A 1 when the consumer says they agree with the statement: 'Offers quality I can trust.'"
    "\n- Cares about the environment: A 1 when the consumer says they agree with the statement: 'Cares about the environment.'"
    "\n- Is easy to find in stores: A 1 when the consumer says they agree with the statement: 'Is easy to find in stores.'"
    "\n- Has a unique taste: A 1 when the consumer says they agree with the statement: 'Has a unique taste.'"
    "\n- Gives me an energy boost: A 1 when the consumer says they agree with the statement: 'Gives me an energy boost.'"
    "\n- Goes well with food: A 1 when the consumer says they agree with the statement: 'Goes well with food.'"
    "\n- Brings me happiness: A 1 when the consumer says they agree with the statement: 'Brings me happiness.'"
    "\n- Is available in convenient packaging: A 1 when the consumer says they agree with the statement: 'Is available in convenient packaging.'"
    "\n- Makes moments special: A 1 when the consumer says they agree with the statement: 'Makes moments special.'"
    "\n- Has a refreshing taste: A 1 when the consumer says they agree with the statement: 'Has a refreshing taste.'"
    "\n- Helps keep me going: A 1 when the consumer says they agree with the statement: 'Helps keep me going.'"
    "\n- Has the right amount of calories: A 1 when the consumer says they agree with the statement: 'Has the right amount of calories.'"
    "\n- Gives me confidence to be myself: A 1 when the consumer says they agree with the statement: 'Gives me confidence to be myself.'"
    
    "\nBehavior (Diary) Feature Definitions:"
    "\n- DayPart_Name: The general day part (time of day) the respondent had their drink: Early morning (between 6am and 8am), Mid-Morning (between 8am and 11am), Midday (between 11am and 2pm), etc."
    "\n- Where_Name: A feature identifying where the product was consumed."
    "\n- Who_Group_HL: A feature identifying whether the person consumed the product alone, with others, etc."
    "\n- Bought_Name: A feature identifying whether the person taking the survey bought the product they consumed or if someone else purchased it."
    "\n- IC/FC-SS/MS: A calculated column that classifies user-reported drinks into combinations of future-consumption (larger containers than what they can consume in one sitting), immediate consumption (smaller containers ready to drink), single-serve (individual containers for immediate consumption), and multiserve (multiple containers sold together, like 12 pack of cans) based on reported purchase pack and reported container drank from size."
    "\n- Channel: A higher-level purchase channel (and level 2 of the channel hierarchy) such as Supermarket/Hypermarket/Warehouse, Grocery Store, Convenience Store/Gas/Petrol Station, etc."
    "\n- Reason_Energize: A 1 when the consumer says this is the reason for drinking the brand: Energize."
    "\n- Reason_Celebrate: A 1 when the consumer says this is the reason for drinking the brand: Celebrate."
    "\n- Reason_Stay: A 1 when the consumer says this is the reason for drinking the brand: Stay."
    "\n- Reason_Rehydrate: A 1 when the consumer says this is the reason for drinking the brand: Rehydrate."
    "\n- Reason_Cheer: A 1 when the consumer says this is the reason for drinking the brand: Cheer."
    "\n- Reason_Close: A 1 when the consumer says this is the reason for drinking the brand: Close."
    "\n- Reason_Cool: A 1 when the consumer says this is the reason for drinking the brand: Cool."
    "\n- Reason_Nutritious: A 1 when the consumer says this is the reason for drinking the brand: Nutritious."
    "\n- Reason_Focus: A 1 when the consumer says this is the reason for drinking the brand: Focus."
    "\n- Reason_Complement: A 1 when the consumer says this is the reason for drinking the brand: Complement."
    "\n- Reason_Confident: A 1 when the consumer says this is the reason for drinking the brand: Confident."
    "\n- Reason_Taste: A 1 when the consumer says this is the reason for drinking the brand: Taste."
    "\n- Reason_Reward: A 1 when the consumer says this is the reason for drinking the brand: Reward."
    "\n- Reason_Restore: A 1 when the consumer says this is the reason for drinking the brand: Restore."
    "\n- Reason_Performance: A 1 when the consumer says this is the reason for drinking the brand: Performance."
    "\n- Reason_Other: A 1 when the consumer says this is the reason for drinking the brand: Other."
    "\n- High-Level-Occasions: The key drinking occasions such as: Being Productive, Snacking, Eating, Active Leisure/Exercise, Relaxing, Routine Behaviors, Starting The Day, Socializing."

    "\nThings in general to know about these products/categories:"
    "\n- Grocery, large store, club store, etc. may be more likely to sell multipacks, which have been shown to drive frequency of consumption when consumers can stock up and have product at home."
    "\n- Intenders is a function of Affinity, Meets Needs, and Consideration. Intenders are people who we believe are highly likely of becoming weekly+."
    "\n- This data is for Coca-Cola Original in the context of the Sparkling Soft Drinks category in the United States."
    "\n- Users prefer answers like the following:"
    "\n- Familiarity: Being familiar with the brand significantly influences consumption frequency. Increasing familiarity with the drink can elevate the likelihood of weekly consumption by 13.8%—making this the most impactful variable."
    "\n- vs"
    "\n- **Familiarity_Name_Drink most often**, - Change in predicted probability: +13.8%, - Building familiarity with your brand is crucial."
    "\n\n- Including a strategic recommendation/guidance at the end will be appreciated by the marketer who wants to know what they should do."
    "\n- Reference descriptions of variables (like Self-Purchase) over actual variable names (Bought_Name) when possible."

    "\n\n DO NOT USE ANY OF THIS DATA FROM THIS EXAMPLE DIRECTLY IN YOUR ANSWER. ONLY USE THIS AS A GUIDE FOR HOW TO STRUCTURE YOUR RESPONSE:"
    "\n"
    """BEGINNING OF EXAMPLE:
    To enhance the frequency of consumption for Coca-Cola Original within the Sparkling Soft Drinks category, here are the critical features that significantly influence weekly consumption, along with their respective impacts on the predicted probability of frequency.

        ### Key Drivers for Increasing Frequency of Consumption:
        
        1. **Familiarity with the Brand**  
           Increasing familiarity can boost likelihood of weekly consumption by 13.8%. Marketers should focus on improving brand recognition and recall to drive higher engagement.
        
        2. **Affinity Towards the Brand**  
           Positive feelings towards the brand contribute an increase of 7.4% in weekly consumption likelihood. Implementing campaigns that foster emotional connections can be beneficial.
        
        3. **Intenders Segment**  
           Targeting consumers who are likely to become weekly consumers can raise probabilities by 5.8%. Marketers should identify and engage these potential customers proactively.
        
        4. **Brand Preference**  
           When consumers indicate the brand would be their first choice, it can raise weekly consumption likelihood by 5.4%. Strategies focusing on making the brand a top-of-mind option are essential.
        
        5. **Meeting Consumer Needs**  
           Highlighting how well Coca-Cola meets consumer needs can increase frequency by 4.8%. Marketers should align product messaging with key consumer requirements.
        
        ### Behavioral Insights:
        
        1. **Self-Purchase**  
           Encouraging consumers to buy Coca-Cola themselves can increase weekly consumption likelihood by 5.0%. Promotions and incentives for self-purchase should be emphasized.
        
        2. **Consumption at Home**  
           Positioning Coca-Cola as a staple at home can boost frequency by 2.9%. Retail strategies focused on home consumption occasions, like multi-pack offers, would be effective.
        
        3. **Independent Stores**  
           Ensuring availability at non-chain grocery stores can lead to a 2.1% increase in frequency. Building relationships with these retailers might enhance accessibility.
        
        4. **Single-Serve Options**  
           Offering single-serve packaging can increase chances of weekly consumption by 1.6%. This can cater to consumers looking for convenient drinking options.
        
        5. **Eating Occasions**  
           Associating the drink with meals can enhance frequency by 0.9%. Marketing strategies should leverage mealtime consumption opportunities.
        
        ### Recommendations:
        - Focus on building brand familiarity through targeted advertising and community engagement.
        - Develop campaigns that emphasize emotional connections and the unique qualities of Coca-Cola.
        - Use promotions to encourage self-purchase and availability in grocery stores, especially independent ones.
        - Position Coca-Cola effectively for home consumption and highlight its refreshing taste and energizing attributes. 
        
        By strategically implementing these insights, there is potential for significant growth in the frequency of Coca-Cola consumption among consumers.
        END OF EXAMPLE
        """
    "\n\nPlease summarize the following results for a marketer trying to grow their brand's frequency. Format all percentages with only 1 decimal point. Remove Asterisks/Bolding references in final response."
    "\n\n\n\n DATA AND INSIGHTS TO BE USED IN YOUR RESPONSE:"

)
prompt = f"{prompt_structure}\n\n{context}\n\n{diaryprompt}\n\n{diary_insight}\n\n{equityprompt}\n\n{equity_insight}\n\n"

print(prompt)


I have results from a model aimed at helping marketers understand how to make people drink their brands every single week based on Diary (behavior) and Equity (attitudes/perceptions) data. The following prompt is structured as follows:
- A summary of context about the question to be answered, meaning of variables, and an example output that can help guide the structure of the final response.
- The results of the diary-specific random forest model that help explain what makes people drink every single week (weekly+).
- A few key insights based on the Diary-specific random forest results.
- The results of the equity-specific random forest model that help explain what makes people drink every single week (weekly+).
- A few key insights based on the equity-specific random forest results.
- Always structure the response with three sections split with the following headers: '#### Behavior Factors', '#### Equity Factors', '#### Recommendations'


CONTEXT, DEFINITIONS, AND EXAMPLES: Below are 

#### Final qualitative review from ChatGPT

In [65]:
# Prepare the final prompt to send to ChatGPT
prompt = f"{prompt_structure}\n\n{context}\n\n{diaryprompt}\n\n{diary_insight}\n\n{equityprompt}\n\n{equity_insight}\n\n"

# Chat completion request payload
irequest = {
    "temperature": 1,
    "top_p": 1,
    "max_tokens": 1000,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
}

# Convert request to JSON
data = json.dumps(irequest)

# Define request URL and headers
url = f"https://apim-emt-aip-prod-01.azure-api.net/openai/deployments/{deployment}/chat/completions?api-version={apiversion}"
hdr = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key
}

# Make the request
try:
    req = urllib.request.Request(url, headers=hdr, data=bytes(data.encode("utf-8")))
    req.get_method = lambda: 'POST'
    response = urllib.request.urlopen(req)
    
    # Parse the response and extract content
    response_body = response.read()
    response_json = json.loads(response_body.decode('utf-8'))
    
    # Extract and print only the content of the response
    response_content = response_json['choices'][0]['message']['content']
    print(response_content)

except Exception as e:
    print(f"An error occurred: {e}")

#### Behavior Factors

The following variables significantly increase the likelihood of weekly consumption of the brand:

1. **Self-Purchase**
   When buyers purchase the product themselves, the probability of weekly consumption increases by 5.1%.

2. **Consumption at Home**
   When the product is consumed at home, the likelihood of weekly consumption rises by 3.7%.

3. **Independent Store Availability**
   The availability of the product at non-chain, independent stores contributes to a 2.0% increase in the likelihood of weekly consumption.

4. **Supermarket/Hypermarket/Warehouse Store Availability**
   The availability of the product at supermarkets, hypermarkets, and warehouse stores can increase the probability of weekly consumption by 1.9%.

5. **Immediate Consumption Single-Serve Products (IC-SS)**
   Providing users with immediate consumption, single-serve options can increase the likelihood of weekly consumption by 1.6%.

6. **Midday Consumption (between 11am and 2pm)**
   When

## Final Output
Output Generation via PowerPoint

#### Extract the correct rows from GPT response and store them into variables for use in PowerPoint

In [66]:
# Function to extract specific sections from the content
def extract_sections(content):
    # Split the content by lines
    lines = content.splitlines()
    
    # Initialize empty strings for each section
    behavior_factors = []
    equity_factors = []
    recommendations = []
    
    # Initialize flags to track the current section
    in_behavior = False
    in_equity = False
    in_recommendations = False
    
    # Loop through each line and categorize it into the appropriate section
    for line in lines:
        line = line.strip()  # Remove any extra spaces
        
        # Detect section headers and toggle flags
        if "#### Behavior Factors" in line:
            in_behavior = True
            in_equity = False
            in_recommendations = False
        elif "#### Equity Factors" in line:
            in_behavior = False
            in_equity = True
            in_recommendations = False
        elif "#### Recommendations" in line:
            in_behavior = False
            in_equity = False
            in_recommendations = True
        else:
            # Append the line to the appropriate section
            if in_behavior:
                behavior_factors.append(line)
            elif in_equity:
                equity_factors.append(line)
            elif in_recommendations:
                recommendations.append(line)
    
    # Join the lists back into strings
    consumption_drivers_text = "\n".join([line for line in behavior_factors if line]).strip()
    equity_drivers_text = "\n".join([line for line in equity_factors if line]).strip()
    recommendations_text = "\n".join([line for line in recommendations if line]).strip()
    
    return consumption_drivers_text, equity_drivers_text, recommendations_text

# Extract the text from the response content
consumption_drivers_text, equity_drivers_text, recommendations_text = extract_sections(response_content)

# Print the results to check
#print(consumption_drivers_text)
#print(equity_drivers_text)
print(recommendations_text)

With these insights, the following strategic recommendations are offered:
- Encourage consumers to purchase the product themselves and consume it at home.
- Ensure product availability in non-chain grocery stores, supermarkets, hypermarkets, and warehouse stores.
- Offer immediate consumption, single-serve options to cater to a variety of consumption occasions.
- Highlight the brand's role in making everyday activities, like meals and starting the day, more enjoyable.
- Build familiarity and love for the brand among consumers.
- Engage consumers who are predisposed to becoming weekly consumers.
- Show how well the brand meets consumer needs and point out its uniqueness.
- Reinforce the value of the product for its price.


#### Adjust the length of each driver text so that it fits in the text box

Recommendations Text Length Adjustment

In [67]:
new_prompt = f"Reduce the number of lines in the following string to only include 5 recommendations. Accomplish this by reducing the number of recommendations, starting with the last one:\n{recommendations_text}"

# Chat completion request payload
irequest = {
    "temperature": 1,
    "top_p": 1,
    "max_tokens": 1000,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": new_prompt}
    ]
}

# Convert request to JSON
data = json.dumps(irequest)

# Define request URL and headers
url = f"https://apim-emt-aip-prod-01.azure-api.net/openai/deployments/{deployment}/chat/completions?api-version={apiversion}"
hdr = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key
}

# Make the request
try:
    req = urllib.request.Request(url, headers=hdr, data=bytes(data.encode("utf-8")))
    req.get_method = lambda: 'POST'
    response = urllib.request.urlopen(req)
    
    # Parse the response and extract content
    response_body = response.read()
    response_json = json.loads(response_body.decode('utf-8'))
    
    # Extract and print only the content of the response
    response_content = response_json['choices'][0]['message']['content']
    print(response_content)

except Exception as e:
    print(f"An error occurred: {e}")

recommendations_text = response_content

With these insights, the following strategic recommendations are offered:
- Encourage consumers to purchase the product themselves and consume it at home.
- Ensure product availability in non-chain grocery stores, supermarkets, hypermarkets, and warehouse stores.
- Highlight the brand's role in making everyday activities, like meals and starting the day, more enjoyable.
- Build familiarity and love for the brand among consumers.
- Engage consumers who are predisposed to becoming weekly consumers.


Consumption Drivers Text Length Adjustment

In [72]:
new_prompt = f"Reduce the number of lines in the following response to only include the top 5 drivers in the numbered list.\n{consumption_drivers_text}"

# Chat completion request payload
irequest = {
    "temperature": 1,
    "top_p": 1,
    "max_tokens": 1000,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": new_prompt}
    ]
}

# Convert request to JSON
data = json.dumps(irequest)

# Define request URL and headers
url = f"https://apim-emt-aip-prod-01.azure-api.net/openai/deployments/{deployment}/chat/completions?api-version={apiversion}"
hdr = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key
}

# Make the request
try:
    req = urllib.request.Request(url, headers=hdr, data=bytes(data.encode("utf-8")))
    req.get_method = lambda: 'POST'
    response = urllib.request.urlopen(req)
    
    # Parse the response and extract content
    response_body = response.read()
    response_json = json.loads(response_body.decode('utf-8'))
    
    # Extract and print only the content of the response
    response_content = response_json['choices'][0]['message']['content']
    cleaned_response = "\n".join(line for line in response_content.splitlines() if line.strip())
    print(cleaned_response)

except Exception as e:
    print(f"An error occurred: {e}")

consumption_drivers_text = cleaned_response

Here are the top 5 drivers that significantly increase the likelihood of weekly consumption of the brand:
1. **Self-Purchase**
When buyers purchase the product themselves, the probability of weekly consumption increases by 5.1%.
2. **Consumption at Home**
When the product is consumed at home, the likelihood of weekly consumption rises by 3.7%.
3. **Independent Store Availability**
The availability of the product at non-chain, independent stores contributes to a 2.0% increase in the likelihood of weekly consumption.
4. **Supermarket/Hypermarket/Warehouse Store Availability**
The availability of the product at supermarkets, hypermarkets, and warehouse stores can increase the probability of weekly consumption by 1.9%.
5. **Immediate Consumption Single-Serve Products (IC-SS)**
Providing users with immediate consumption, single-serve options can increase the likelihood of weekly consumption by 1.6%.


Equity Drivers Text Length Adjustment

In [73]:
new_prompt = f"Reduce the number of lines in the following response to only include the top 5 drivers in the numbered list.\n{equity_drivers_text}"

# Chat completion request payload
irequest = {
    "temperature": 1,
    "top_p": 1,
    "max_tokens": 1000,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": new_prompt}
    ]
}

# Convert request to JSON
data = json.dumps(irequest)

# Define request URL and headers
url = f"https://apim-emt-aip-prod-01.azure-api.net/openai/deployments/{deployment}/chat/completions?api-version={apiversion}"
hdr = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key
}

# Make the request
try:
    req = urllib.request.Request(url, headers=hdr, data=bytes(data.encode("utf-8")))
    req.get_method = lambda: 'POST'
    response = urllib.request.urlopen(req)
    
    # Parse the response and extract content
    response_body = response.read()
    response_json = json.loads(response_body.decode('utf-8'))
    
    # Extract and print only the content of the response
    response_content = response_json['choices'][0]['message']['content']
    cleaned_response = "\n".join(line for line in response_content.splitlines() if line.strip())
    print(cleaned_response)

except Exception as e:
    print(f"An error occurred: {e}")

equity_drivers_text = cleaned_response

The top 5 drivers that significantly increase the likelihood of weekly consumption of the brand are:
1. **Familiarity**
When users are familiar with the product, it increases the likelihood of weekly consumption by 11.0%.
2. **Affinity**
Users who love the brand contribute to an 8.8% increase in the likelihood of weekly consumption.
3. **Target Intenders Segment**
Targeting potential users can increase weekly consumption likelihood by 8.2%.
4. **Brand Preference (First Choice)**
Being a consumer's first choice brand can increase the likelihood of weekly consumption by 5.5%.
5. **Meeting Needs**
If consumers believe the brand meets their needs, it can increase weekly consumption likelihood by 4.0%.


Create a string for the Brand + Country Name

In [74]:
brand_string = country_id + " - " + brand_name

#### Get the Brand Logo Image

In [ ]:
import requests
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO

def insert_image_from_url(slide, image_url, left, top, width=None, height=None):
    """Download image from URL and insert into slide at specified position."""
    response = requests.get(image_url)
    
    if response.status_code == 200:
        # Download the image
        image_stream = BytesIO(response.content)

        # Add the image to the slide
        if width and height:
            slide.shapes.add_picture(image_stream, left, top, width, height)
        else:
            slide.shapes.add_picture(image_stream, left, top)

# Loop through slides and shapes to update the text and insert images
for slide in prs.slides:
    for shape in slide.shapes:
        if not shape.has_text_frame:
            continue  # Skip non-text shapes
        
        # Update text in various boxes
        if "recommendations_text" in shape.text:
            update_shape_text_with_bold(shape, recommendations_text, font_size)
        
        if "consumption_drivers_text" in shape.text:
            update_shape_text_with_bold(shape, consumption_drivers_text, font_size)
        
        if "equity_drivers_text" in shape.text:
            update_shape_text_with_bold(shape, equity_drivers_text, font_size)
        
        if "brand_country" in shape.text:
            update_shape_text_with_bold(shape, brand_string, font_size, is_header=True)
    
    # Insert image into a specific slide (adjust slide number as needed)
    # Example URL of an image and its position
    image_url = 'https://example.com/path/to/image.jpg'
    insert_image_from_url(slide, image_url, margin_right=Inches(.25), margin_top=Inches.25))

# Save the updated PowerPoint presentation
prs.save(output_path)

print(f"PowerPoint saved as {output_path}")


#### Create a PowerPoint Presentation with this information

In [75]:
# Load the template PowerPoint file
prs = Presentation(template_path)

# Define the desired font sizes (in points)
font_size = Pt(11)  # Regular font size
font_size_header = Pt(20)  # Header font size

def update_shape_text_with_bold(shape, new_text, font_size, is_header=False):
    # Clear existing text
    text_frame = shape.text_frame
    text_frame.clear()

    # Find parts of the text that need to be bold
    parts = re.split(r"(\*\*.*?\*\*)", new_text)  # Split by the bold markers '**'

    # Add a single paragraph to the text frame
    p = text_frame.add_paragraph()
    
    # Add the text in parts, formatting as necessary
    for part in parts:
        if part.startswith("**") and part.endswith("**"):
            # Remove the '**' markers and set the text as bold
            bold_text = part[2:-2]
            run = p.add_run()
            run.text = bold_text
            run.font.bold = True
        else:
            # Non-bold text
            run = p.add_run()
            run.text = part
        
        # Set the font size and style
        if is_header:
            run.font.size = font_size_header
        else:
            run.font.size = font_size

def update_shape_as_numbered_list(shape, text, font_size):
    # Clear existing text
    text_frame = shape.text_frame
    text_frame.clear()

    # Split the text into items assuming each item is separated by a newline
    items = text.strip().split('\n')

    for i, item in enumerate(items):
        # Add a new paragraph for each item
        p = text_frame.add_paragraph()
        
        # Add the number and format it with font size
        num_run = p.add_run()
        num_run.text = f"{i + 1}. "  # Numbering
        num_run.font.size = font_size  # Set font size for the number
        
        # Process the item text for bold formatting
        parts = re.split(r"(\*\*.*?\*\*)", item.strip())  # Split by the bold markers '**'
        
        for part in parts:
            if part.startswith("**") and part.endswith("**"):
                # Remove the '**' markers and set the text as bold
                bold_text = part[2:-2]
                run = p.add_run()
                run.text = bold_text
                run.font.bold = True
            else:
                # Non-bold text
                run = p.add_run()
                run.text = part
            
            # Set the font size for the text
            run.font.size = font_size

def insert_image_from_url(slide, image_url, left, top, width=None, height=None):
    """Download image from URL and insert into slide at specified position."""
    response = requests.get(image_url)
    
    if response.status_code == 200:
        # Download the image
        image_stream = BytesIO(response.content)

        # Add the image to the slide
        if width and height:
            slide.shapes.add_picture(image_stream, left, top, width, height)
        else:
            slide.shapes.add_picture(image_stream, left, top)

# Loop through slides and shapes to update the text
for slide in prs.slides:
    for shape in slide.shapes:
        if not shape.has_text_frame:
            continue  # Skip non-text shapes
        
        # Update text in various boxes
        if "recommendations_text" in shape.text:
            update_shape_text_with_bold(shape, recommendations_text, font_size)
        
        if "consumption_drivers_text" in shape.text:
            update_shape_text_with_bold(shape, consumption_drivers_text, font_size)
        
        if "equity_drivers_text" in shape.text:
            update_shape_text_with_bold(shape, equity_drivers_text, font_size)
        
        if "brand_country" in shape.text:
            update_shape_text_with_bold(shape, brand_string, font_size, is_header=True)
    
    # Insert image into a specific slide (adjust slide number as needed)
    # Example URL of an image and its position
    image_url = 'https://example.com/path/to/image.jpg'
    insert_image_from_url(slide, image_url, margin_right=Inches(.25), margin_top=Inches.25))

# Save the updated PowerPoint presentation
prs.save(output_path)

print(f"PowerPoint saved as {output_path}")

PowerPoint saved as C:\Users\O52834\OneDrive - The Coca-Cola Company\Documents\BEACH\W+ Modeling\Iteration Outputs\USA_br_0201.pptx
